### Notebook -> Entendimento Completo do algoritmo de clustering

In [0]:
import numpy as np
from sklearn.metrics import silhouette_score
from sklearn.datasets import load_wine
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from scipy.spatial import ConvexHull
from scipy.spatial import distance
from sklearn.decomposition import PCA


In [0]:
dataset = load_wine().data
features=load_wine().feature_names
X=pd.DataFrame(dataset,columns=features)
X.head()

In [0]:
class Kmeans:
    def __init__(self,k,iterations=300,tol=10e-4):
        self.k=k
        self.iterations=iterations
        self.tol=tol
    def centroides_iniciais(self):
        """Função para definicão inicial dos centróides, vamos utilizar o metodo aleatorio de inicio"
        """
        #escolha dos centróides
        idxs = np.random.choice(dataset.shape[0],self.k,replace=False)
        centroides = dataset[idxs, :]    
        return centroides
    def calcula_J(self,dataset,centroides):
        """Função para calcular a menor distancia entre os pontos e os centroides
        parametros de entrada : dataset e os centroides_iniciais
        Retorno: Menores distancias e menor J"""
        diff =dataset[:,None,:]-centroides[None,:,:]
        d=np.sum(diff**2,axis=2)
        j=np.sum(np.min(d,axis=1))
        menores=np.argmin(d,axis=1)
        return menores,j,d
    def atualiza_centroides(self,dataset,menores,k):
        "Função que irá fazer o cálculo dos novos centróides"
        "Parametros de entrada: dataset, menores e k"
        "Retorno: novos centroides"
        lists=[]
        for i in range(k):
            grupo=dataset[menores==i]
            grupo=np.mean(grupo,axis=0)
            lists.append(grupo)
            centroides=np.array(lists)
        return centroides

    def run_kmeans(self,X,centroides, tol, max_iter):
        """Função para calcular a função de custo
        Parametro de entrada: DATASET E OS CENTROIDES tolerancia e max iterações opcionais
        retorno: Valor de J(Função de custo)"""
        c=centroides
        labels,j,d= self.calcula_J(X,c)
        centroides=self.atualiza_centroides(X,labels,self.k)
        labels,j_final,d=self.calcula_J(X,centroides)
        it=0
        while abs(j-j_final)>tol and it<max_iter:    
            labels=np.argmin(d,axis=1)
            centroides=self.atualiza_centroides(X,labels,self.k)
            j=j_final
            labels,j_final,d=self.calcula_J(X,centroides)
            it+=1
        return j,labels,centroides




    

In [0]:
#Chama o PCA para reduzir a dimensionalidade
pca = PCA(n_components=2)
dataset = pca.fit_transform(X)
tol=1e-4
max_iter=300
#Chama a função Kmeans
kmeans=Kmeans(k=3)
centroides=kmeans.centroides_iniciais()
j,labels,centroides=kmeans.run_kmeans(dataset,centroides,tol,max_iter)





In [0]:
#plotando a solução com os centroides
fig,ax =plt.subplots(figsize=(10,6))
colors=['blue','red','green']
k= centroides.shape[0]
for i in range(k):
    plt.scatter(dataset[labels==i,0],dataset[labels==i,1],label=f'Cluster {i}',color=colors[i])
    if dataset[labels == i].shape[0] >= 3:
        
        hull = ConvexHull(dataset[labels == i],qhull_options='QJ')  
        poly = dataset[labels == i][hull.vertices]
        
        poly = np.vstack([poly, poly[0]])
        
        ax.plot(poly[:, 0], poly[:, 1], linewidth=2)
        
        ax.fill(poly[:, 0], poly[:, 1], alpha=0.08)
    ax.scatter(centroides[i, 0], centroides[i, 1], marker='o')
    plt.scatter(centroides[:,0],centroides[:,1],c='black')
plt.title("Dados")
plt.xlabel("X")
plt.ylabel("Y")
plt.legend()
ax.grid(True, alpha=0.25)
plt.show()

